[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google/vizier/blob/main/docs/tutorials/automl_conf_2023.ipynb)

# AutoML-Conf 2023 Industry Day Tutorial: Open Source Vizier

## Schedule
1. Basics
2. Algorithm API
3. Benchmark API

In [ ]:
# Installation
!pip install google-vizier[jax]

# 1. Basics ([Reference](https://oss-vizier.readthedocs.io/en/latest/guides/user/running_vizier.html))

Suppose we want to figure out the optimal amount of chocolate to use in a cookie recipe to maximize taste, and the relationship is:
$$ \text{taste} = 1 - 2(\text{chocolate} - 0.3)^{2}$$ where `chocolate` is within $[0,1]$.

**Exercise: Implement a problem statement and evaluation function for the optimization setup above.**

In [ ]:
# @title Solution (Hidden)
from vizier.service import pyvizier as vz

problem = vz.ProblemStatement()
problem.search_space.root.add_float_param('chocolate', 0.0, 1.0)
problem.metric_information.append(
    vz.MetricInformation(
        name='taste', goal=vz.ObjectiveMetricGoal.MAXIMIZE))

def evaluate(chocolate: float) -> float:
    return 1 - 2 * (chocolate - 0.3)**2

**Exercise: Use our default optimizer over this objective and print the trajectory over 10 trials.**

In [ ]:
# @title Solution (Hidden)
from vizier.service import clients

study_config = vz.StudyConfig.from_problem(problem)
study_config.algorithm = 'GAUSSIAN_PROCESS_BANDIT'
study_client = clients.Study.from_study_config(study_config, owner='my_name', study_id='cookie_recipe')

for _ in range(10):
  suggestions = study_client.suggest(count=1)
  for suggestion in suggestions:
    chocolate = suggestion.parameters['chocolate']
    obj = evaluate(chocolate)
    print(f'Iteration {suggestion.id}, suggestion (chocolate={chocolate:.3f}) led to taste value {obj:.3f}.')
    final_measurement = vz.Measurement({'taste': obj})
    suggestion.complete(final_measurement)

**Exercise: Use the client to automatically obtain the best trial so-far and all historical trials.**

In [ ]:
# @title Solution (Hidden)
optimal_trial_client = list(study_client.optimal_trials())[0]
optimal_trial = optimal_trial_client.materialize()

all_trials = [tc.materialize() for tc in study_client.trials()]

To visually understand what's going on, we'd better plot the following:

1. Objective function curve.

2. Historical trials (xy-points)

3. Optimal trial (highlighted xy-point)


**Exercise: Use the results of previous cells to plot the above in a single figure.**



In [ ]:
# @title Solution (Hidden)
import numpy as np
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))

# Visualize the real objective function.
xs = np.linspace(0.0, 1.0, num=1000)
ys = [evaluate(x) for x in xs]
plt.plot(xs, ys, label='actual', color='blue', alpha=0.6)

# Visualize all trials so-far.
trial_xs = [t.parameters['chocolate'].value for t in all_trials]
trial_ys = [evaluate(x) for x in trial_xs]
plt.scatter(trial_xs, trial_ys, label='trials', marker='o', color='red')

# Mark optimal trial so far.
optimal_trial_xs = [optimal_trial.parameters['chocolate'].value]
optimal_trial_ys = [evaluate(x) for x in optimal_trial_xs]
plt.scatter(optimal_trial_xs, optimal_trial_ys, label='optimal', marker='x', color='green', s = 100)

# Plot.
plt.legend()
plt.title(f'Chocolate vs Taste')
plt.xlabel('chocolate')
plt.ylabel('taste')

plt.show()

Of course, our recipe doesn't only include chocolate. For example, we may need to optimize other ingredients, such as salt and sugar. Suppose our new taste objective is defined as:
$$ \text{taste} = 1 - (\text{chocolate} - 0.2)^{2} - (\text{salt} - 0.5)^{2} - (\text{sugar} - 0.3)^{2}$$ where `chocolate`, `salt`, `sugar` are all within $[0,1]$.

Also, suppose we wanted to try a different algorithm, such as random search, to quickly and comprehensively cover the search space.

**Exercise: How would we setup a new study (problem statement, algorithm) for this problem? This time, loop over 100 trials.**


In [ ]:
# @title Solution (Hidden)
problem = vz.ProblemStatement()
problem.search_space.root.add_float_param('chocolate', 0.0, 1.0)
problem.search_space.root.add_float_param('salt', 0.0, 1.0)
problem.search_space.root.add_float_param('sugar', 0.0, 1.0)
problem.metric_information.append(
    vz.MetricInformation(
        name='taste', goal=vz.ObjectiveMetricGoal.MAXIMIZE))

study_config = vz.StudyConfig.from_problem(problem)
study_config.algorithm = 'RANDOM_SEARCH'
study_client = clients.Study.from_study_config(study_config, owner='my_name', study_id='new_cookie_recipe')

def evaluate(chocolate: float, salt: float, sugar: float) -> float:
    return 1 - (chocolate - 0.2)**2 - (salt - 0.5)**2 - (sugar - 0.3)**2

for _ in range(100):
  suggestions = study_client.suggest(count=1)
  for suggestion in suggestions:
    chocolate = suggestion.parameters['chocolate']
    salt = suggestion.parameters['salt']
    sugar = suggestion.parameters['sugar']
    obj = evaluate(chocolate, salt, sugar)
    print(f'Iteration {suggestion.id}, suggestion (chocolate={chocolate:.2f}, salt={salt:.2f}, sugar={sugar:.2f}) led to taste value {obj:.2f}.')
    final_measurement = vz.Measurement({'taste': obj})
    suggestion.complete(final_measurement)

Our kitchen can simultaneously try out different recipes. This means we can *batch* our evaluations using multiple clients.

**Exercise: Using `multiprocessing.pool.ThreadPool`, construct multiple clients to parallelize evaluations on a single machine.**

In [ ]:
# @title Solution (Hidden)
import multiprocessing

NUM_CLIENTS = 10
NUM_TRIALS_PER_CLIENT = 50

def thread_fn(client_id: int):
  thread_client = clients.Study.from_resource_name(study_client.resource_name)

  for _ in range(NUM_TRIALS_PER_CLIENT):
    suggestions = thread_client.suggest(count=1, client_id=str(client_id))
    for suggestion in suggestions:
      chocolate = suggestion.parameters['chocolate']
      salt = suggestion.parameters['salt']
      sugar = suggestion.parameters['sugar']

      obj = evaluate(chocolate, salt, sugar)
      final_measurement = vz.Measurement({'taste': obj})
      suggestion.complete(final_measurement)

pool = multiprocessing.pool.ThreadPool(NUM_CLIENTS)
pool.map(thread_fn, range(NUM_CLIENTS))

Suppose we wanted full control over our server in order to be able to edit the database location, or the list of algorithms supported.

**Exercise: Create a server explicitly and have a fresh client connect to this server.**

In [ ]:
# @title Solution (Hidden)
from vizier.service import servers
server = servers.DefaultVizierServer()
clients.environment_variables.server_endpoint = server.endpoint
study_client = clients.Study.from_study_config(study_config, owner='owner', study_id='cookie_recipe_new_server')

# 2. Algorithm API ([Reference](https://oss-vizier.readthedocs.io/en/latest/guides/index.html#for-developers))

Let's take a closer look at the algorithms optimizing our cookie recipes. Let's use Grid Search as an example. Run the setup below.

In [ ]:
# @title Required Import
from vizier._src.algorithms.designers import grid as grid_lib
from vizier.service import pyvizier as vz

search_space = vz.SearchSpace()
search_space.root.add_float_param('chocolate', 0.0, 1.0)

grid_designer = grid_lib.GridSearchDesigner(search_space)

**Exercise: Let's use `grid_designer` to generate the first 5 suggestion. Print them.**

In [ ]:
# @title Solution (Hidden)
suggestions = grid_designer.suggest(count=5)
for s in suggestions:
  print(s)

GridSearch is a `PartiallySerializableDesigner`, which allows the serialization and de-serlization of the Designer and its current state. The internal state of `grid_designer` can be obtained via `dump()`. States are of form `vz.Metadata` ([Reference](https://oss-vizier.readthedocs.io/en/latest/guides/developer/metadata.html)).

**Exercise: Let's inspect the output of `dump()`. What two variables control the state?**

NOTE: Do NOT use `print()`.

In [ ]:
# @title Solution (Hidden)
metadata = grid_designer.dump()
metadata

We can change the designer's internal state via `load()`.

**Exercise: Modify the metadata state to use a grid position. Verify by calling the designer's `suggest()` again.**

NOTE: You will need to use `metadata.ns()` to change namespaces.

In [ ]:
# @title Solution (Hidden)
metadata.ns('grid')['current_index'] = '7'
grid_designer.load(metadata)

print(grid_designer.suggest()[0])

If we wanted to use `GridSearchDesigner` as a hosted service policy, we'll need to wrap it as a `Policy` and then write a `PolicyFactory` for the service.

**Exercise: Create a subclass `CustomPolicyFactory` to accomodate our grid designer.** ([Reference](https://github.com/google/vizier/blob/main/vizier/_src/service/policy_factory.py))

In [ ]:
# @title Solution (Hidden)
from vizier import pythia
from vizier import algorithms as vza

class CustomPolicyFactory(pythia.PolicyFactory):

  def __call__(
      self,
      problem_statement: vz.ProblemStatement,
      algorithm: str,
      policy_supporter: pythia.PolicySupporter,
      study_name: str,
  ) -> pythia.Policy:
    """Creates a Pythia Policy."""

    if algorithm == 'GRID_SEARCH':
      from vizier._src.algorithms.designers import grid

      return vza.PartiallySerializableDesignerPolicy(
          problem_statement,
          policy_supporter,
          grid.GridSearchDesigner.from_problem,
      )

We'll need to host our new `CustomPolicyFactory`.

**Exercise: Recreate a new Vizier Server to use our new `CustomPolicyFactory`.**

In [ ]:
# @title Solution (Hidden)
from vizier.service import servers

custom_policy_factory = CustomPolicyFactory()
new_server = servers.DefaultVizierServer(policy_factory=custom_policy_factory)

# 3. Benchmark API ([Reference](https://oss-vizier.readthedocs.io/en/latest/guides/index.html#for-benchmarking))

Many times grid search isn't the most efficient algorithm, especially in high dimensions. We'll need to benchmark different algorithms in a systematic way.

In [ ]:
# @title Required Imports
from vizier import benchmarks as vzb
from vizier.benchmarks import experimenters
from vizier._src.algorithms.designers import random as random_lib

Let's formalize our cookie objective as an `Experimenter` class, which contains an evaluation method and problem statement ([Reference](https://github.com/google/vizier/blob/main/vizier/benchmarks/experimenters/__init__.py)).

```python
class Experimenter(metaclass=abc.ABCMeta):
  """Abstract base class for Experimenters."""

  @abc.abstractmethod
  def evaluate(self, suggestions: Sequence[vz.Trial]) -> None:
    """Evaluates and mutates the Trials in-place."""

  @abc.abstractmethod
  def problem_statement(self) -> vz.ProblemStatement:
    """The search configuration generated by this experimenter."""
```

NOTE: The Experimenter evaluates and completes the `Trial` in place.

**Exercise: Create a custom `CookieExperimenter` for our cookie objective.**

In [ ]:
# @title Solution (Hidden)

class CookieExperimenter(experimenters.Experimenter):

  def evaluate(self, suggestions) -> None:
    """Evaluates and mutates the Trials in-place."""
    for suggestion in suggestions:
      chocolate = suggestion.parameters['chocolate'].value
      salt = suggestion.parameters['salt'].value
      sugar = suggestion.parameters['sugar'].value
      taste =  1 - (chocolate - 0.2)**2 - (salt - 0.5)**2 - (sugar - 0.3)**2
      suggestion.complete(vz.Measurement({'taste': taste}))

  def problem_statement(self) -> vz.ProblemStatement:
    problem = vz.ProblemStatement()
    problem.search_space.root.add_float_param('chocolate', 0.0, 1.0)
    problem.search_space.root.add_float_param('salt', 0.0, 1.0)
    problem.search_space.root.add_float_param('sugar', 0.0, 1.0)
    problem.metric_information.append(
        vz.MetricInformation(
            name='taste', goal=vz.ObjectiveMetricGoal.MAXIMIZE))
    return problem


Let's test how the `CookieExperimenter` works using the code below.

In [ ]:
# @title Test for Exercise
experimenter = CookieExperimenter()
trial = vz.Trial(parameters={'chocolate': 0.5, 'salt': 0.5, 'sugar': 0.5})
experimenter.evaluate([trial])
assert trial.final_measurement.metrics['taste'].value == 0.87

Now let's run two different algorithms with `CookieExperimenter`. Conceptually, every study is just a simple loop between an algorithm (`Designer`/`Policy`) and objective (`Experimenter`).

We use `BenchmarkRunner` routines to specify how to run and modify a `BenchmarkState`, which holds information about the objective via an `Experimenter` and the algorithm itself wrapped by a `PolicySuggester`.

```python
class BenchmarkState:
  """State of a benchmark run. It is altered via benchmark protocols."""

  experimenter: Experimenter
  algorithm: PolicySuggester
```

**Exercise: Initialize BenchmarkStates with Random and Grid Designers.**

NOTE: Use `BenchmarkState` factories provided [here](https://github.com/google/vizier/blob/main/vizier/_src/benchmarks/runners/benchmark_state.py).

In [ ]:
# @title Solution (Hidden)
benchmark_states = []

grid_designer_factory = grid_lib.GridSearchDesigner.from_problem
grid_state_factory = vzb.DesignerBenchmarkStateFactory(
    experimenter=experimenter, designer_factory=grid_designer_factory
)
benchmark_states.append(grid_state_factory())

random_designer_factory = random_lib.RandomDesigner.from_problem
random_state_factory = vzb.DesignerBenchmarkStateFactory(
    experimenter=experimenter, designer_factory=random_designer_factory
)
benchmark_states.append(random_state_factory())

In [ ]:
# @title Test for Exercise
runner = vzb.BenchmarkRunner(
      benchmark_subroutines=[
          vzb.GenerateSuggestions(),
          vzb.EvaluateActiveTrials(),
      ],
      num_repeats=100,
  )
for benchmark_state in benchmark_states:
  runner.run(benchmark_state)
  assert len(benchmark_state.algorithm.supporter.GetTrials()) == 100

We can convert `BenchmarkState`s to primitive data formats using our [analysis library](https://github.com/google/vizier/blob/main/vizier/benchmarks/analyzers.py). For single-objectives, we extract and plot the objective metric, which represents the objective of the best `Trial` seen so far.

In [ ]:
# @title Analysis Demonstration
from vizier.benchmarks import analyzers
import matplotlib.pyplot as plt

for idx, benchmark_state in enumerate(benchmark_states):
  curve = analyzers.BenchmarkStateAnalyzer.to_curve([benchmark_state])
  plt.plot(curve.xs, curve.ys.flatten(), label = f'algo {idx}')
plt.legend()
plt.xlabel('Number of trials')
plt.ylabel('Objective value')
plt.show()


Finally, we show the flexibility of our basic setup. In a few lines of code, we can accomplish the following with relative ease:

* Add a noisy Cookie benchmark
* Add discretization to the chocolate Cookie parameter
* Add normalized metrics for analysis
* Add another algorithm for comparison
* Add repeats and error bars

In [ ]:
from vizier._src.algorithms.designers.eagle_strategy import eagle_strategy

NUM_REPEATS = 5  # @param
NUM_ITERATIONS = 100  # @param

algorithms = {
    'grid': grid_lib.GridSearchDesigner.from_problem,
    'random': random_lib.RandomDesigner.from_problem,
    'eagle': eagle_strategy.EagleStrategyDesigner,
}


class CookieExperimenterFactory(experimenters.SerializableExperimenterFactory):

  def __call__(self, *, seed=None) -> experimenters.Experimenter:
    return CookieExperimenter()

  def dump(self):
    return vz.Metadata({'name': 'CookieExperimenter'})


experimenter_factories = [
    CookieExperimenterFactory(),
    experimenters.SingleObjectiveExperimenterFactory(
        base_factory=CookieExperimenterFactory(),
        noise_type='SEVERE_ADDITIVE_GAUSSIAN',
    ),
    experimenters.SingleObjectiveExperimenterFactory(
        base_factory=CookieExperimenterFactory(),
        discrete_dict = {0: 4}
    )
]

records = []
for experimenter_factory in experimenter_factories:
  for algo_name, algo_factory in algorithms.items():
    benchmark_state_factory = vzb.ExperimenterDesignerBenchmarkStateFactory(
        experimenter_factory=experimenter_factory, designer_factory=algo_factory
    )
    states = []
    for _ in range(NUM_REPEATS):
      benchmark_state = benchmark_state_factory()
      runner.run(benchmark_state)
      states.append(benchmark_state)
    record = analyzers.BenchmarkStateAnalyzer.to_record(
        algorithm=algo_name,
        experimenter_factory=experimenter_factory,
        states=states,
    )
    records.append(record)

analyzed_records = analyzers.BenchmarkRecordAnalyzer.add_comparison_metrics(
    records=records, baseline_algo='random'
)
analyzers.plot_from_records(analyzed_records, title_maxlen=100, col_figsize=12)